### Exploring the data structures of MEMBRANES, and reactions in them 
#### - with NO DIFFUSION

LAST REVISED: May 28, 2023

In [1]:
import set_path      # Importing this module will add the project's home directory to sys.path

Added 'D:\Docs\- MY CODE\BioSimulations\life123-Win7' to sys.path


In [2]:
from src.modules.reactions.reaction_data import ChemData as chem
from src.modules.reactions.reaction_dynamics import ReactionDynamics

from src.life_1D.bio_sim_1d import BioSim1D

In [3]:
chem_data = chem(names=["A", "B", "C"])     # NOTE: Diffusion not done
bio = BioSim1D(n_bins=5, chem_data=chem_data)

bio.set_membranes(membrane_pos=[1])   # A single membrane, passing thru bin 1

bio.set_all_uniform_concentrations(conc_list=[4., 8., 12.])

bio.describe_state()

SYSTEM STATE at Time t = 0:
5 bins and 3 species:
  Species 0 (A). Diff rate: None. Conc: |4.0|4.0()4.0|4.0|4.0|4.0|
  Species 1 (B). Diff rate: None. Conc: |8.0|8.0()8.0|8.0|8.0|8.0|
  Species 2 (C). Diff rate: None. Conc: |12.0|12.0()12.0|12.0|12.0|12.0|


In [4]:
bio.set_bin_conc(bin_address=1, species_name="A", conc=10.)
bio.set_bin_conc(bin_address=1, species_name="A", conc=55., across_membrane=True)
bio.set_bin_conc(bin_address=1, species_name="B", conc=20.)
bio.set_bin_conc(bin_address=1, species_name="C", conc=30., both_sides=True)

bio.describe_state()

SYSTEM STATE at Time t = 0:
5 bins and 3 species:
  Species 0 (A). Diff rate: None. Conc: |4.0|10.0()55.0|4.0|4.0|4.0|
  Species 1 (B). Diff rate: None. Conc: |8.0|20.0()8.0|8.0|8.0|8.0|
  Species 2 (C). Diff rate: None. Conc: |12.0|30.0()30.0|12.0|12.0|12.0|


In [5]:
# Make the last bin match all the concentrations of the "post-membrane" section of bin 1
bio.set_bin_conc(bin_address=4, species_name="A", conc=55.)
bio.set_bin_conc(bin_address=4, species_name="C", conc=30.)

bio.describe_state()

SYSTEM STATE at Time t = 0:
5 bins and 3 species:
  Species 0 (A). Diff rate: None. Conc: |4.0|10.0()55.0|4.0|4.0|55.0|
  Species 1 (B). Diff rate: None. Conc: |8.0|20.0()8.0|8.0|8.0|8.0|
  Species 2 (C). Diff rate: None. Conc: |12.0|30.0()30.0|12.0|12.0|30.0|


In [6]:
# Reaction A + B <-> C , with 1st-order kinetics in both directions, mostly forward
bio.chem_data.add_reaction(reactants=["A", "B"], products=["C"], forward_rate=8., reverse_rate=2.)

bio.chem_data.describe_reactions()

Number of reactions: 1 (at temp. 25 C)
0: A + B <-> C  (kF = 8 / kR = 2 / Delta_G = -3,436.56 / K = 4) | 1st order in all reactants & products


In [7]:
bio.react(time_step=0.002, n_steps=1)
bio.describe_state()

SYSTEM STATE at Time t = 0.002:
5 bins and 3 species:
  Species 0 (A). Diff rate: None. Conc: |3.536|6.92()48.08|3.536|3.536|48.08|
  Species 1 (B). Diff rate: None. Conc: |7.536|16.92()1.08|7.536|7.536|1.08|
  Species 2 (C). Diff rate: None. Conc: |12.464|33.08()36.92|12.464|12.464|36.92|


### Note how (in the absence of diffusion, which we're neglecting) the concentrations on the "post-membrane side" of bin 1 continue to match those of bin 5

## Now continue to reaction equilibrium

In [8]:
bio.react(time_step=0.002, n_steps=100)
bio.describe_state()

SYSTEM STATE at Time t = 0.202:
5 bins and 3 species:
  Species 0 (A). Diff rate: None. Conc: |0.7932534523016194|0.8970947370566021()47.20020986266437|0.7932534523016194|0.7932534523016194|47.20020986266437|
  Species 1 (B). Diff rate: None. Conc: |4.793253452301623|10.897094737056594()0.20020986266437912|4.793253452301623|4.793253452301623|0.20020986266437912|
  Species 2 (C). Diff rate: None. Conc: |15.206746547698396|39.10290526294337()37.79979013733563|15.206746547698396|15.206746547698396|37.79979013733563|


### The system has now reached equilibrium
### in individual bins, which remain separate because we're NOT doing diffusion in this experiment

Verify the equilibrium in each of the active bins

In [9]:
bio.reaction_dynamics.is_in_equilibrium(rxn_index=0, conc={"A": 0.7932534523016195, "B": 4.793253452301622, "C": 15.206746547698396})
# A was largely the limiting reagent

A + B <-> C
Final concentrations:  [C] = 15.21 ; [A] = 0.7933 ; [B] = 4.793
1. Ratio of reactant/product concentrations, adjusted for reaction orders: 3.99939
    Formula used:  [C] / ([A][B])
2. Ratio of forward/reverse reaction rates: 4.0
Discrepancy between the two values: 0.01521 %
Reaction IS in equilibrium (within 1% tolerance)



True

In [10]:
bio.reaction_dynamics.is_in_equilibrium(rxn_index=0, conc={"A": 0.897094737056602, "B": 10.897094737056594, "C": 39.10290526294337})
# A was largely the limiting reagent

A + B <-> C
Final concentrations:  [C] = 39.1 ; [A] = 0.8971 ; [B] = 10.9
1. Ratio of reactant/product concentrations, adjusted for reaction orders: 4
    Formula used:  [C] / ([A][B])
2. Ratio of forward/reverse reaction rates: 4.0
Discrepancy between the two values: 2.259e-07 %
Reaction IS in equilibrium (within 1% tolerance)



True

In [11]:
bio.reaction_dynamics.is_in_equilibrium(rxn_index=0, conc={"A": 47.20020986266437, "B": 0.20020986266437912, "C": 37.79979013733563})
# This time, with ample [A], the limiting reagent was B

A + B <-> C
Final concentrations:  [C] = 37.8 ; [A] = 47.2 ; [B] = 0.2002
1. Ratio of reactant/product concentrations, adjusted for reaction orders: 4
    Formula used:  [C] / ([A][B])
2. Ratio of forward/reverse reaction rates: 4.0
Discrepancy between the two values: 0 %
Reaction IS in equilibrium (within 1% tolerance)



True